In [2]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention
from tensorflow.keras.models import Model

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
def transformer_encoder(inputs, num_heads=4, ff_dim=64, dropout=0.1):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    x = LayerNormalization()(inputs + attention)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return LayerNormalization()(inputs + x) 

In [5]:
def create_market_transformer(seq_length=30, num_features=4, num_heads=4, ff_dim=64):
    inputs = Input(shape=(seq_length, num_features))
    
    x = transformer_encoder(inputs, num_heads, ff_dim)
    x = transformer_encoder(x, num_heads, ff_dim)
    
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    
    return Model(inputs, x)

In [6]:
market_model = create_market_transformer()
market_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [7]:
market_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 4)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 30, 4)             │             308 │ input_layer[0][0],         │
│ (MultiHeadAttention)          │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 30, 4)             │               0 │ input_layer[0][0],         │
│                               │                           │                 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 30, 4)             │               8 │ add[0][0]                  │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 30, 64)            │             320 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 30, 64)            │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 30, 4)             │             260 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 30, 4)             │               0 │ input_layer[0][0],         │
│                               │                           │                 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 30, 4)             │               8 │ add_1[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_1        │ (None, 30, 4)             │             308 │ layer_normalization_1[0][… │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_2 (Add)                   │ (None, 30, 4)             │               0 │ layer_normalization_1[0][… │
│                               │                           │                 │ multi_head_attention_1[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_2         │ (None, 30, 4)             │               8 │ add_2[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 30, 64)            │             320 │ layer_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 2,001 (7.82 KB)

 Trainable params: 2,001 (7.82 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
def create_ticker_transformer(seq_length=30, num_features=4, num_heads=4, ff_dim=64, num_tickers=30, embed_dim=8):
    # Ticker Input (Embedding)
    ticker_input = Input(shape=(), dtype=tf.int32, name="ticker_input")
    ticker_embedding = tf.keras.layers.Embedding(input_dim=num_tickers, output_dim=embed_dim)(ticker_input)

    # Features Input
    features_input = Input(shape=(seq_length, num_features), name="features_input")

    # Expand embedding to match sequence length
    expanded_embedding = tf.keras.layers.RepeatVector(seq_length)(ticker_embedding)

    # Concatenate ticker embedding with features
    x = tf.keras.layers.Concatenate()([features_input, expanded_embedding])

    # Transformer Encoder
    x = transformer_encoder(x, num_heads, ff_dim)
    x = transformer_encoder(x, num_heads, ff_dim)
    
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(1, activation="linear")(x) # Predicting next day's `Pct_Change`

    return Model([ticker_input, features_input], x)

In [9]:
ticker_model = create_ticker_transformer(num_tickers=30)
ticker_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [44]:
ticker_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ticker_input (InputLayer)     │ (None)                    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 8)                 │             240 │ ticker_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ features_input (InputLayer)   │ (None, 30, 4)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ repeat_vector (RepeatVector)  │ (None, 30, 8)             │               0 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 30, 12)            │               0 │ features_input[0][0],      │
│                               │                           │                 │ repeat_vector[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_2        │ (None, 30, 12)            │           2,460 │ concatenate[0][0],         │
│ (MultiHeadAttention)          │                           │                 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_4 (Add)                   │ (None, 30, 12)            │               0 │ concatenate[0][0],         │
│                               │                           │                 │ multi_head_attention_2[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_4         │ (None, 30, 12)            │              24 │ add_4[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 30, 64)            │             832 │ layer_normalization_4[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_5 (Dropout)           │ (None, 30, 64)            │               0 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 30, 12)            │             780 │ dropout_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_5 (Add)                   │ (None, 30, 12)            │               0 │ concatenate[0][0],         │
│                               │                           │                 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_5         │ (None, 30, 12)            │              24 │ add_5[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_3        │ (None, 30, 12)            │           2,460 │ layer_normalization_5[0][… │
│ (MultiHeadAttention)          │                           │               

 Total params: 26,789 (104.65 KB)

 Trainable params: 8,929 (34.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,860 (69.77 KB)

In [11]:
market_index = pd.read_csv('market_index.csv', index_col='Date')
ticker_features = pd.read_csv('ticker_features.csv', index_col=["Date", "Name"])

In [12]:
market_scaler = StandardScaler()
X_market = market_scaler.fit_transform(market_index[['Pct_Change', 'MA_5', 'MA_10', 'Volatility']].values)

In [13]:
def create_sequences(data, sequence_length=30):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length, 0])
    return np.array(X), np.array(y)

In [14]:
X_market_seq, y_market_seq = create_sequences(X_market)

In [15]:
ticker_scaler = StandardScaler()
X_ticker = ticker_scaler.fit_transform(ticker_features[['Pct_Change', 'MA_5', 'MA_10', 'Volume_MA_5']].values)

In [16]:
label_encoder = LabelEncoder()
ticker_ids = label_encoder.fit_transform(ticker_features.index.get_level_values('Name'))

In [32]:
print(f"X_market_seq shape: {X_market_seq.shape}")
print(f"y_market_seq shape: {y_market_seq.shape}")  

X_market_seq shape: (2980, 30, 4)
y_market_seq shape: (2980,)


In [48]:
X_train, X_val, y_train, y_val = train_test_split(X_market_seq, y_market_seq, test_size=0.1, random_state=42)

In [50]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32).prefetch(tf.data.AUTOTUNE)

In [54]:
market_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 26s 57ms/step - loss: 0.9372 - mae: 0.7255 - val_loss: 1.0563 - val_mae: 0.7573
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.9178 - mae: 0.7157 - val_loss: 1.0516 - val_mae: 0.7573
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.9221 - mae: 0.7171 - val_loss: 1.0460 - val_mae: 0.7513
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.9146 - mae: 0.7142 - val_loss: 1.0563 - val_mae: 0.7564
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.9154 - mae: 0.7150 - val_loss: 1.0534 - val_mae: 0.7546
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.9129 - mae: 0.7137 - val_loss: 1.0539 - val_mae: 0.7547
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.9137 - mae: 0.7143 - val_loss: 1.0505 - val_mae: 0.7526
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.9114 - mae: 0.7130 - val_loss: 1.0511 - val_mae: 0.7542
Epoch 9/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.91

In [17]:
def create_ticker_sequences(ticker_features, sequence_length=30):
    X, y, ids = [], [], []
    
    for ticker, group in ticker_features.groupby(level='Name'):
        group = group.sort_index(level='Date')  # Ensure dates are in order
        data = ticker_scaler.transform(group[['Pct_Change', 'MA_5', 'MA_10', 'Volume_MA_5']].values)
        ticker_id = label_encoder.transform([ticker])[0]  # Encode the ticker name

        for i in range(len(data) - sequence_length):
            X.append(data[i:i+sequence_length])
            y.append(data[i+sequence_length, 0])  # Predicting Pct_Change
            ids.append(ticker_id)  # Append ticker ID

    return np.array(X), np.array(y), np.array(ids)

# Create sequences with IDs
X_ticker_seq, y_ticker_seq, ticker_id_seq = create_ticker_sequences(ticker_features)

In [18]:
X_train2, X_val2, y_train2, y_val2, id_train2, id_val2 = train_test_split(X_ticker_seq, y_ticker_seq, ticker_id_seq, test_size=0.1, random_state=42)

In [40]:
train_ticker_dataset = tf.data.Dataset.from_tensor_slices(
    ({"features_input": X_train2, "ticker_input": id_train2}, y_train2)
)
train_ticker_dataset = train_ticker_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

val_ticker_dataset = tf.data.Dataset.from_tensor_slices(
    ({"features_input": X_val2, "ticker_input": id_val2}, y_val2)
)
val_ticker_dataset = val_ticker_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [42]:
ticker_model.fit(train_ticker_dataset, epochs=20, validation_data=val_ticker_dataset)

Epoch 1/20


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [46]:
tf.keras.backend.clear_session()

In [48]:
ticker_features

,,Pct_Change,MA_5,MA_10,Volume_MA_5
Date,Name,,,,
2006-01-17,AABA,0.005263,41.150,41.579,26959826.8
2006-01-18,AABA,-0.122912,39.590,41.006,47436189.4
2006-01-19,AABA,-0.024161,38.082,40.342,54381322.2
2006-01-20,AABA,-0.017186,36.652,39.563,62133322.6
2006-01-23,AABA,0.012745,35.506,38.659,62118938.6
...,...,...,...,...,...
2017-12-22,XOM,0.001431,83.214,83.091,10801680.6
2017-12-26,XOM,0.000119,83.422,83.186,9884430.6
2017-12-27,XOM,-0.000953,83.714,83.300,9227542.6
